# FFDI and Thresholding

code is setup to process the intial bias_input_data into FFDI and thresholding.

The FFDI code uses the zarr stores of the individual simulations which requires the rechunking

Thresholding uses the FFDI files of the individual simulations

Setup to provide both average value for a GWL for a given RCM and values for each year of a GWL

In [1]:
import sys
import os

import intake
import xarray as xr
from matplotlib import pyplot as plt
import glob
import pathlib
import traceback
from sys import argv
from datetime import datetime
from xclim.indices import (
    keetch_byram_drought_index,
    griffiths_drought_factor,
    mcarthur_forest_fire_danger_index
)
from dask.distributed import Client        
import dask
import warnings

# Needed for the GWL code
from importlib import reload
# adding folder to the system path
sys.path.insert(0, '/g/data/xv83/rxm599/acs/gwls')

import gwl

In [2]:
def setup_dask_client():
# Set configuration options
    dask.config.set({
    'distributed.comm.timeouts.connect': '90s',  # Timeout for connecting to a worker
    'distributed.comm.timeouts.tcp': '90s',  # Timeout for TCP communications
    })
    client = Client()
    return client

In [3]:
def extract_model_info(filepath):
    filename = filepath.split('/')[-1]  # Get the filename from the full path
    filename_no_ext = filename.split('.')[0]  # Remove the file extension
    parts = filename_no_ext.split('_')  # Split filename by underscores
#    print(parts)
    RCM=parts[1]
    GCM=parts[2]
    
    # Extract model name
    model_name = filename_no_ext
    extension = filename.split('.')[-1]  # Get the extension
    
    match = '_'.join(parts[1:3])  # Extract match (e.g., EC-Earth3_ssp370_r1i1p1f1)
    pathway = parts[3]  # Extract model (e.g., ssp370)
    ensemble = parts[4]  # Extract ensemble (e.g., r1i1p1f1)
    
    return [model_name, extension], RCM, GCM, ensemble, pathway

In [4]:
def process_threshold(data, time_dim, syear, eyear, var, period):
# first set time slice
    start=str(syear)+'-01-01'
    end=str(eyear)+'-12-31'
    nyear=eyear-syear+1; inyear=1./nyear
    data = data.sel(**{time_dim: slice(start, end)}).persist() # use file chunking .persist()
    print(f"Processing period {period}")
# days over key thresholds (days/yr)
    d3 = (data > 100).sum('time').to_dataset(name='days100') *inyear
    d2 = (data > 75).sum('time').to_dataset(name='days75') *inyear
    d1 = (data > 50).sum('time').to_dataset(name='days50') *inyear
    dss=xr.merge([d1,d2,d3]) 
    return dss,data

In [5]:
def process_thresholda(data, time_dim, syear, eyear, var, period):
# first set time slice
    start=str(syear)+'-01-01'
    end=str(eyear)+'-12-31'
    nyear=eyear-syear+1; inyear=1./nyear
    data = data.sel(**{time_dim: slice(start, end)}).persist() # use file chunking .persist()
    print(f"Processing period {period}")
# days over key thresholds (days/yr)
#    d100 = data > 100 ; d100.groupby(time_dim.year).mean
    d3a=(data > 100 ).groupby('time.year').sum('time').to_dataset(name='days100')
    d2a=(data > 75 ).groupby('time.year').sum('time').to_dataset(name='days75')
    d1a=(data > 50 ).groupby('time.year').sum('time').to_dataset(name='days50')
    d4a=(data > 25 ).groupby('time.year').sum('time').to_dataset(name='days25')
    d3 = (data > 100).sum('time').to_dataset(name='days100') *inyear
    d2 = (data > 75).sum('time').to_dataset(name='days75') *inyear
    d1 = (data > 50).sum('time').to_dataset(name='days50') *inyear
    d4 = (data > 25).sum('time').to_dataset(name='days25') *inyear
    dss=xr.merge([d1,d2,d3,d4]) 
    dsa=xr.merge([d1a,d2a,d3a,d4a]) 
    return dss,data,dsa

In [9]:
def process_GWL_levels(path_ffdi,model_name, GCM, ensemble, pathway, ffdi_data):
    CMIP='CMIP6'
    GWL_levels = ['1.2', '1.5', '2.0', '3.0']
#    GWL_levels = ['1.2']
# historical pathway is skipped
    for GWL in GWL_levels:
        print(f"Processing GWL {GWL} for model: {GCM}, ensemble: {ensemble}, pathway: {pathway}")
        try:
            start, end = gwl.get_GWL_syear_eyear(CMIP, GCM, ensemble, pathway,GWL= GWL)
            print(f'year range= {start},{end} for {GWL}')
        except:
            print(f"No GWL data found for {GWL}")
            d4=0
            continue
# append the historical data to the ssp dataset before doing thresholding
        if pathway != 'historical' :
            mm=model_name.replace(pathway, "historical")
            ffdi0=xr.open_zarr(path_ffdi+mm+'_FFDI.zarr')
            ffdi1=xr.open_zarr(path_ffdi+model_name+'_FFDI.zarr')
            ffdi2=ffdi1.sel(time=slice('2015-01-01','2099-12-31') )
            ffdi_data=xr.concat([ffdi0,ffdi2],dim='time')
            print(ffdi_data)
        else :
            ffdi_data=xr.open_zarr(path_ffdi+model_name+'_FFDI.zarr')
        
        d4, ddata, d4a = process_thresholda(ffdi_data.FFDI, 'time', start, end, 'FFDI', GWL)
#        d4, ddata = process_threshold(ffdi_data.FFDI, 'time', start, end, 'FFDI', GWL)
        output_path = f'/scratch/xv83/rxm599/tmp/{model_name}_GWL{GWL}_ffdi_threshold.nc'
        output_path2= f'/scratch/xv83/rxm599/tmp/{model_name}_GWL{GWL}_ffdi_thresholda.nc'
        output_path1 = f'/scratch/xv83/rxm599/tmp/{model_name}_GWL{GWL}_ffdi.zarr'
        print(f"Saved  to {output_path}")
        d4.to_netcdf(output_path)
        d4a.to_netcdf(output_path2)
#        ddata.to_zarr(output_path1)
    return d4

In [7]:
# Example code to read from command line if not interactive (just make the cell code)
def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')


In [10]:
%%time
def main():
    warnings.filterwarnings('ignore')
# start dask client
    client = setup_dask_client()
    qtype='adjust'
    kind='QME-BARRAR2'
#    kind='EC-Earth3-Veg'
# iterate through all modesl
    if is_interactive():
        print(f"Dashboard available at: {client.dashboard_link}")
    else:
        print(argv)
        print(argv)
# raw or adjust
        qtype=argv[1]
        kind=argv[2]

    if qtype == 'raw' :
        catalogue_path = '/g/data/ia39/catalogues/bias-input'
        path_ffdi='/g/data/ia39/ncra/fire/bias-input/ffdi/'
    elif qtype == 'adjust' :
        catalogue_path = '/g/data/ia39/catalogues/bias-output'
        path_ffdi='/g/data/ia39/ncra/fire/bias-output/ffdi/'
        
# get RCMs to processes
#    mRuns = sorted(glob.glob(catalogue_path + '/*ssp126*.json'))
    mRuns = sorted(glob.glob(catalogue_path + '/*'+kind+'*.json'))
    print(len(mRuns))
        
    print(mRuns)
    for mindex, file in enumerate(mRuns):
        print(file)
        model_name, RCM, GCM, ensemble, pathway = extract_model_info(file)
        var='FFDI'  # not used
        if mindex != 991:         #this model failed to convert
            print (mindex)
            d4=process_GWL_levels(path_ffdi,model_name[0],GCM,ensemble,pathway,var)
    
    print("Processing of all catalogues is complete.")

    print(f"close client {client}")
    client.close()

if __name__ == "__main__":
    main()

Modifying workers


/g/data/hh5/public/apps/dask-optimiser/schedplugin.py:160: UserWarning: Attempted illegal binding: []
  warn(f"Attempted illegal binding: {(sorted(list(self.slots_available - taken_slots))[:self.worker.state.nthreads])}")
/g/data/hh5/public/apps/dask-optimiser/schedplugin.py:160: UserWarning: Attempted illegal binding: []
  warn(f"Attempted illegal binding: {(sorted(list(self.slots_available - taken_slots))[:self.worker.state.nthreads])}")
/g/data/hh5/public/apps/dask-optimiser/schedplugin.py:160: UserWarning: Attempted illegal binding: []
  warn(f"Attempted illegal binding: {(sorted(list(self.slots_available - taken_slots))[:self.worker.state.nthreads])}")
/g/data/hh5/public/apps/dask-optimiser/schedplugin.py:160: UserWarning: Attempted illegal binding: []
  warn(f"Attempted illegal binding: {(sorted(list(self.slots_available - taken_slots))[:self.worker.state.nthreads])}")
/g/data/hh5/public/apps/dask-optimiser/schedplugin.py:160: UserWarning: Attempted illegal binding: []
  warn(f"A

Dashboard available at: /node/gadi-cpu-bdw-0198.gadi.nci.org.au/60389/proxy/42365/status
117
['/g/data/ia39/catalogues/bias-output/AUST-05i_BOM_ACCESS-CM2_historical_r4i1p1f1_BARPA-R_v1-r1-ACS-QME-BARRAR2-1980-2022_day.json', '/g/data/ia39/catalogues/bias-output/AUST-05i_BOM_ACCESS-CM2_ssp126_r4i1p1f1_BARPA-R_v1-r1-ACS-QME-BARRAR2-1980-2022_day.json', '/g/data/ia39/catalogues/bias-output/AUST-05i_BOM_ACCESS-CM2_ssp370_r4i1p1f1_BARPA-R_v1-r1-ACS-QME-BARRAR2-1980-2022_day.json', '/g/data/ia39/catalogues/bias-output/AUST-05i_BOM_ACCESS-ESM1-5_historical_r6i1p1f1_BARPA-R_v1-r1-ACS-QME-BARRAR2-1980-2022_day.json', '/g/data/ia39/catalogues/bias-output/AUST-05i_BOM_ACCESS-ESM1-5_ssp126_r6i1p1f1_BARPA-R_v1-r1-ACS-QME-BARRAR2-1980-2022_day.json', '/g/data/ia39/catalogues/bias-output/AUST-05i_BOM_ACCESS-ESM1-5_ssp370_r6i1p1f1_BARPA-R_v1-r1-ACS-QME-BARRAR2-1980-2022_day.json', '/g/data/ia39/catalogues/bias-output/AUST-05i_BOM_CESM2_historical_r11i1p1f1_BARPA-R_v1-r1-ACS-QME-BARRAR2-1980-2022_day.

INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: method = None, returning None
INFO:flox:_choose_engine: Choosing 'flox'
INFO:flox:find_group_cohorts: blockwise is preferred.
INFO:flox:_choose_method: method is None
INFO:flox:_choose_method: choosing preferred_method=blockwise
INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: reindex is False
INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: method = None, returning None
INFO:flox:_choose_engine: Choosing 'flox'
INFO:flox:find_group_cohorts: blockwise is preferred.
INFO:flox:_choose_method: method is None
INFO:flox:_choose_method: choosing preferred_method=blockwise
INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: reindex is False
INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: method = None, returning None
INFO:flox:_choose_engin

Saved  to /scratch/xv83/rxm599/tmp/AUST-05i_BOM_ACCESS-CM2_ssp126_r4i1p1f1_BARPA-R_v1-r1-ACS-QME-BARRAR2-1980-2022_day_GWL1.2_ffdi_threshold.nc
Processing GWL 1.5 for model: ACCESS-CM2, ensemble: r4i1p1f1, pathway: ssp126
/g/data/xv83/rxm599/acs/gwls
year range= 2019,2038 for 1.5


...

INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: method = None, returning None
INFO:flox:_choose_engine: Choosing 'flox'
INFO:flox:_choose_method: method is None
INFO:flox:_choose_method: choosing preferred_method=blockwise
INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: reindex is False


Saved  to /scratch/xv83/rxm599/tmp/AUST-05i_BOM_ACCESS-ESM1-5_ssp370_r6i1p1f1_BARPA-R_v1-r1-ACS-QME-BARRAR2-1980-2022_day_GWL1.2_ffdi_threshold.nc


2025-03-11 14:48:08,049 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('groupby_sum-store-map-38ea528e79bfd2f5ea3b53b6671453c5', 7, 11, 0))" coro=<Worker.execute() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/worker_state_machine.py:3615>> ended with CancelledError
2025-03-11 14:48:08,050 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('groupby_sum-store-map-642a9d5e15c79e682b41911bb431f25d', 17, 18, 0))" coro=<Worker.execute() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/worker_state_machine.py:3615>> ended with CancelledError
2025-03-11 14:48:08,050 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('groupby_sum-store-map-9c3f109daa75b2c85a19858f05bb5600', 8, 6, 0))" coro=<Worker.execute() done, d

KeyboardInterrupt: 

2025-03-11 14:48:11,682 - distributed.scheduler - ERROR - Task original-open_dataset-FFDI-4ff7a0d3795d31ec05087ae99a1be92b marked as failed because 4 workers died while trying to run it
2025-03-11 14:48:11,684 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:42367' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('getitem-111c857047c664416740527ca0051fb7', 14, 0, 0), ('store-map-9c3f109daa75b2c85a19858f05bb5600', 13, 14, 0), ('store-map-642a9d5e15c79e682b41911bb431f25d', 12, 6, 0), ('getitem-111c857047c664416740527ca0051fb7', 18, 17, 0), ('store-map-3a16351c0073605f04709cb1dab929b6', 0, 11, 0), ('store-map-642a9d5e15c79e682b41911bb431f25d', 18, 17, 0), ('store-map-9c3f109daa75b2c85a19858f05bb5600', 18, 15, 0), ('store-map-3a16351c0073605f04709cb1dab929b6', 18, 1, 0), ('getitem-111c857047c664416740527ca0051fb7', 3, 5, 0), ('groupby_sum-store-map-9c3f109daa75b2c85a19858f05bb5600', 14, 16, 0), ('store-map-642a9d5e15c79e682b4191